### Figma中存在rect元素，将其全部转为Path

In [1]:
import xml.etree.ElementTree as ET
from svgelements import Rect, Matrix
# 加载SVG文件
def parse_svg(svg_file):
    # 解析SVG文件
    tree = ET.parse(svg_file)
    root = tree.getroot()
    # 命名空间 (一般SVG文件会包含默认的命名空间)
    # 根据具体的SVG文件，可能需要调整命名空间前缀
    namespace = {'svg': 'http://www.w3.org/2000/svg'}
    
    # 查找所有 <g> 元素
    root_g = root.find('./svg:g', namespace)  # 使用命名空间避免解析问题
    g_elements = root_g.findall('./svg:g', namespace)  # 使用命名空间避免解析问题
    
    layers = {}
    # 遍历 <g> 元素并提取属性 （图层）
    for g in g_elements:
        # print(f"Found <g> element with attributes: {g.attrib}")
        elements = []
        # 如果需要进一步提取 <g> （图层） 内子元素及其信息
        for child in g:
            childTag = child.tag.replace('{http://www.w3.org/2000/svg}','')
            if childTag == "rect":
                rect_attr = child.attrib
                rect_tranform = rect_attr.get("transform", "")
                if rect_tranform:
                    d = Rect(rect_attr['x'], rect_attr['y'], rect_attr['width'], rect_attr['height'], transform=rect_tranform).d()
                else:
                    d = Rect(rect_attr['x'], rect_attr['y'], rect_attr['width'], rect_attr['height']).d()
                del rect_attr['x']
                del rect_attr['y']
                del rect_attr['width']
                del rect_attr['height']
                rect_attr['d'] = d
                elements.append({**rect_attr, 'tag': 'path'})
                continue
            elements.append({**child.attrib, 'tag': childTag})
        layers[g.attrib['id']] = elements
    return layers

# 示例SVG文件路径
svg_file = '../public/Changan.svg'  # 替换为你的SVG文件路径
layers = parse_svg(svg_file)


In [2]:
import json
with open("../public/TangData.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(layers, ensure_ascii=False))